In [1]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# read ROIs from geojson file

In [2]:
import json

roiFile=r".\files\Multi_Class_Land_Cover_Change_AOIs.geojson"

with open(roiFile) as f:
    data = json.load(f)
    
for feature in data['features']:
    #print(feature['properties'])
    #featureGEE=ee.Feature(geojsonDict)
    
    '''convert to gee feature'''
    #jsonDict = eval(geom.to_json())
    #geojsonDict = jsonDict['features'][0]

    cor=feature['geometry']['coordinates']
         
    aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
    #print(aoi)

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# visualize one aoi

In [9]:
feature =data['features'][0]
cor=feature['geometry']['coordinates']     
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)

print(feature['properties']['title'])

print(cor[0][0][0], cor[0][0][1])


2832_4366_13
-55.546875 -11.824341483849047


In [28]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,10)
Map.addLayer(aoi, {'color': 'FF0000'})

# download monthly s2 data for one aoi 

In [26]:
timePeriod = ee.DateRange('2018-01-01', '2018-02-01')
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
#['Blue', 'Green', 'Red', 'Red Edge 1', 'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4', 'SWIR1', 'SWIR2', 'QA60']

import os
saveFolder= r".\files\res"
filename = os.path.join(saveFolder, feature['properties']['title']+'_201801'+'.tif')

In [14]:
def get_utm_epsg(lon, lat):
	#coords = feature.geometry().coordinates()
	lon = ee.Number(lon)
	lat = ee.Number(lat)
	epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
	#print(epsg.getInfo())
	return ee.String("EPSG:").cat(ee.String(str(epsg.getInfo())))#ee.String("EPSG:").cat

In [17]:
#ee.ImageCollection("COPERNICUS/S2_SR")
#https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
               
def get_s1_mosaic(city):
    #input is a gee feature
    
    roi = city.geometry()

    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(timePeriod).filterBounds(roi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(s1_bands) \
    .map(clip_roi) \
    .median()
    return collection
def getS2(aoi, timePeriod):
	# Define function to mask out clouds and cloud shadows in images.
	# Use CFmask band included in USGS Landsat SR image product.
	def fmask(img):
		cloudOpaqueBitMask = 1 << 10
		cloudCirrusBitMask = 1 << 11
		qa = img.select('QA60')
		mask = qa.bitwiseAnd(cloudOpaqueBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
		return(img.updateMask(mask))
               
	image = ee.ImageCollection('COPERNICUS/S2_SR') \
	.filterDate(timePeriod).select(s2_bands) \
	.filterBounds(aoi).map(fmask).median()

	return image.clip(aoi)


In [27]:
#s2 = getS2(aoi, timePeriod)
s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(timePeriod).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)

#print(s2.getInfo())

'for debug'
#print(s2.getInfo())
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2")

In [19]:
epsg=get_utm_epsg(cor[0][0][0], cor[0][0][1])
print(epsg.getInfo())

EPSG:32721


In [24]:
print(filename[:-4]+".zip")

.\files\res\2832_4366_13_201801.zip


In [25]:
geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
os.remove(filename[:-4]+".zip")

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_201801.zip'


# monthly

In [32]:
#for year in [2018,2019]:

year=2018

for m in range(11,12):
    
    start_date = ee.Date.fromYMD(year, m, 1)
    end_date = ee.Date.fromYMD(year, m+1, 1)
    
    s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(start_date, end_date).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)
    
    filename = os.path.join(saveFolder, feature['properties']['title']+'_'+str(year)+'_'+str(m)+'.tif')
    
    geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
    
    os.remove(filename[:-4]+".zip")

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2018_11.zip'


In [33]:
start_date = ee.Date.fromYMD(year, 12, 1)
end_date = ee.Date.fromYMD(year+1, 1, 1)
    
s2 = ee.ImageCollection('COPERNICUS/S2') \
.filterDate(start_date, end_date).select(s2_bands) \
.filterBounds(aoi).median().clip(aoi)
    
filename = os.path.join(saveFolder, feature['properties']['title']+'_'+str(year)+'_'+str(12)+'.tif')
    
geemap.ee_export_image(s2, filename=filename, scale=10, region=aoi, crs=ee.Projection(epsg), file_per_band=False)
    
os.remove(filename[:-4]+".zip")    

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2018_12.zip'


Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2019_1.zip'
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2019_2.zip'
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2019_3.zip'
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\2832_4366_13_2019_4.zip'
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_